In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pretrainedmodels

import torchvision
from torch.utils.data import Dataset

import torchvision.transforms as transforms

from tqdm import tqdm

from skimage import io
import cv2

In [2]:
path_to_points = 'data/300W/train/134212_1.pts'

In [3]:
def read_keypoint(jpg_path: str) -> torch.FloatTensor:
    #assume .pts file is in the same fir as .jpg
    pts_name = jpg_path[:-4] + '.pts'
    with open(pts_name) as f:
        lines = f.readlines()
        if lines[0].startswith('version'):  # to support different formats
            lines = lines[3:-1]
        mat = np.fromstring(''.join(lines), sep=' ')
        mat_tensor = mat.reshape(-1, 2)
        # visibility = torch.ones([68, 1], dtype=torch.float)
        # keypoint = torch.cat((mat_tensor, visibility), dim=1)                    
    return mat_tensor

In [4]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dir_to_jpgs: str, transform=None):
        """
        Arguments:
            dir_to_folder (string): Path to folder with .jpg and .pts files.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = dir_to_jpgs
        self.resize = 224
        self.transform = transform
        self.images = []
        for idx, fname in enumerate(os.listdir(self.root_dir)):
            cur_path = os.path.join(self.root_dir, fname)
            if cur_path.endswith('.jpg'):
                self.images.append(cur_path)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        keypoint = read_keypoint(img_path) # (np_array[68, 2]) format points : [x, y]
        img = img / 255 # normalize values  
        img_height, img_width, _ = img.shape
        img = cv2.resize(img, (self.resize, self.resize))

        if self.transform is not None:
            img = self.transform(img) # to tensor, from shape (H, W, C) -> (C, H, W)

        keypoint = keypoint * [self.resize / img_width, self.resize / img_height]

        return {'image': img.to(torch.float),
                'keypoints': torch.tensor(keypoint, dtype=torch.float),
                }

    def __len__(self):
        return len(self.images)

In [5]:
path_to_train = 'data/300W/train/'
batch_size = 8

In [6]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = FaceLandmarksDataset(dir_to_jpgs=path_to_train, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [7]:
path_to_val = 'data/300W/val/'

In [8]:
valset = FaceLandmarksDataset(dir_to_jpgs=path_to_val, transform=transform)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)

In [9]:
class FaceKeypointResNet34(nn.Module):
    def __init__(self, pretrained, requires_grad):
        super(FaceKeypointResNet34, self).__init__()
        if pretrained == True:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained=None)
        if requires_grad == True:
            for param in self.model.parameters():
                param.requires_grad = True
            print('Training intermediate layer parameters...')
        elif requires_grad == False:
            for param in self.model.parameters():
                param.requires_grad = False
            print('Freezing intermediate layer parameters...')
        # change the final layer
        self.l0 = nn.Linear(512, 136)
    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        l0 = self.l0(x)
        return l0

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# model 
model = FaceKeypointResNet34(pretrained=False, requires_grad=True).to(device)
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# we need a loss function which is good for regression like SmmothL1Loss ...
# ... or MSELoss
criterion = nn.MSELoss()

/home/timur/anaconda3/envs/face_d/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/timur/anaconda3/envs/face_d/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training intermediate layer parameters...


In [12]:
epochs = 30

In [13]:
def fit(model, dataloader, dataset):
    print('Training')
    model.train()
    train_running_loss = 0.0
    counter = 0
    # calculate the number of batches
    num_batches = int(len(dataset)/batch_size)
    for i, data in tqdm(enumerate(dataloader), total=num_batches):
        counter += 1
        if torch.cuda.is_available():
            image, keypoints = data['image'].to(device), data['keypoints'].to(device)
        # flatten the keypoints
        keypoints = keypoints.view(keypoints.size(0), -1)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, keypoints)
        train_running_loss += loss.item()
 
        loss.backward()
        optimizer.step()

        
    train_loss = train_running_loss/counter
    return train_loss

In [14]:
# validatioon function
def validate(model, dataloader, data, epoch):
    print('Validating')
    model.eval()
    valid_running_loss = 0.0
    counter = 0
    # calculate the number of batches
    num_batches = int(len(data)/batch_size)
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=num_batches):
            counter += 1
            image, keypoints = data['image'].to(device), data['keypoints'].to(device)
            # flatten the keypoints
            keypoints = keypoints.view(keypoints.size(0), -1)
            outputs = model(image)
            loss = criterion(outputs, keypoints)
            valid_running_loss += loss.item()
            # plot the predicted validation keypoints after every...
            # ... predefined number of epochs
            # if (epoch+1) % 1 == 0 and i == 0:
            #     utils.valid_keypoints_plot(image, outputs, keypoints, epoch)
        
    valid_loss = valid_running_loss/counter
    return valid_loss

In [19]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit(model, train_loader, trainset)
    val_epoch_loss = validate(model, val_loader, valset, epoch)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f'Val Loss: {val_epoch_loss:.4f}')

Epoch 1 of 30
Training


284it [03:23,  1.40it/s]                         


Validating


32it [00:22,  1.45it/s]                        


Train Loss: 887.9822
Val Loss: 748.3261
Epoch 2 of 30
Training


...

Train Loss: 219.3835
Val Loss: 386.8026
Epoch 30 of 30
Training


284it [03:11,  1.48it/s]                         


Validating


32it [00:22,  1.43it/s]                        

Train Loss: 211.2629
Val Loss: 423.0014


In [20]:
torch.save(model.state_dict(), 'resnet34_weights_300W_e30_np.pth')

In [21]:
torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, "resnet34_300W_e30_np.pth")

In [27]:
model.load_state_dict(torch.load('resnet34_weights_menpo69p_e50_np.pth'))

/tmp/ipykernel_121088/3105838270.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet34_weights_menpo69p_e50_np.pth'))


<All keys matched successfully>

In [28]:
model.eval()

FaceKeypointResNet34(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [32]:
img_test_path = 'data/300W/test/261068_2.jpg'

In [33]:
img = cv2.cvtColor(cv2.imread(img_test_path), cv2.COLOR_BGR2RGB)
orig_h, orig_w, c = img.shape
img = cv2.resize(img, (224, 224))
img = img / 255.
img = transform(img)
img = img.to(torch.float)
img = img.unsqueeze(0).to(device)


In [34]:
model = model.to(device).eval()
with torch.no_grad():
    outputs = model(img)

In [35]:
outputs = outputs.cpu().detach().numpy()

In [36]:
outputs = outputs.reshape(-1, 2)

In [ ]:
outputs * [orig_w/ 224, orig_h / 224]

In [39]:
np.savetxt('test.pts', outputs, fmt='%.3f', delimiter=' ') 

In [29]:
menpo_68p_test_path = 'data/300W/test'
output_dir = 'data/300W/preds/resnet18_Menpo/'

In [30]:
model = model.to(device).eval()
for idx, fname in enumerate(os.listdir(menpo_68p_test_path)):
    cur_path = os.path.join(menpo_68p_test_path, fname)
    if cur_path.endswith('.jpg') or cur_path.endswith('.png'):
        img = cv2.cvtColor(cv2.imread(cur_path), cv2.COLOR_BGR2RGB)
        orig_h, orig_w, c = img.shape
        img = cv2.resize(img, (224, 224))
        img = img / 255.
        img = transform(img)
        img = img.to(torch.float)
        img = img.unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img)
        outputs = outputs.cpu().detach().numpy()
        outputs = outputs.reshape(-1, 2)
        keypoints = outputs * [orig_w/ 224, orig_h / 224]
        os.makedirs(os.path.dirname(output_dir), exist_ok=True)
        np.savetxt(output_dir + fname[:-4] + '.pts', keypoints, fmt='%.3f', delimiter=' ') 